In [249]:
#Initial Setup: Load libraries
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import os
import re # For regex - not used

import matplotlib.pyplot as plt # from matplotlib import pyplot as plt
import seaborn as sns # statistical graphics in Python - builds on top of matplotlib 
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
# from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import RobustScaler
# from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
# from mlxtend.regressor import StackingCVRegressor
# from xgboost import XGBRegressor
from datetime import datetime

start_time = datetime.now()

In [4]:
#Initial Setup: Load data

print(os.getcwd())
os.chdir('C:\\Users\\rgohi\\Downloads\\')

#Read file
X_full = pd.read_csv('.\\train.csv')
X_test_full = pd.read_csv('.\\test.csv')

C:\Users\rgohi\Documents\Python Scripts


In [ ]:
import sys

da=dir()
sys.getsizeof(pd)

for val in dir():
    print(val, type(eval(val)), sys.getsizeof(val))

In [ ]:
#Data Exploration: Examine data

print(type(X_full)) #datatype
print(len(X_full)) #number of rows
print(X_full.shape) #(row, columns)
print(X_full.columns) #column labels
print(X_full.dtypes.sort_values(axis=0))
print(list(X_full.select_dtypes(include=('float64','int64'))))
print(X_full.info())
print(test.info(memory_usage='deep')) # Can check out memory usage for larger datasets <- can remove observations/features as needed - such as columns with just one value
print(X_full.describe()) #Only displays the numeric columns
print(X_full.describe(include=np.object)) #Only displays the categorical/string columns
print(X_full["LandSlope"].value_counts()) #Displays counts of each group in a columns
print(X_full.loc[X_full["LandSlope"] == "Sev",("LotArea","SalePrice")].agg(("min","max"))) #Displays mix/max of subsets of columns
X_full.head(3)
X_full.tail(3)

In [ ]:
#Data Exploration: Visualize data 1 of 2

print(X_full.iloc[:,:])
# table = pd.pivot_table(data=X_full,index="HouseStyle",values="LotArea",aggfunc=np.average)
# print(table)
# plt.bar(table.index,table["LotArea"])
plt.hist(X_full['YearBuilt'],rwidth=0.9,alpha=0.3,color='blue',bins=15,edgecolor='red') 

plt.boxplot(X_full["YearBuilt"])
# # plt.yticks(range(1, 22))
plt.ylabel("Value")
plt.show()

# sns.relplot(data=X_full,y="LotArea",x="MSZoning",hue="Condition1",size="GarageArea") # kind="line" style ="" # General Plot
print(X_full.columns) #column labels
# sns.lmplot(data=X_full,y="SalePrice",x="LotArea",hue="MSZoning") # Linear Regression
# ? error sns.displot(data=X_full,x="SalePrice",y="LotArea",kde="True")
# ? error sns.catplot(data=X_full,kind="violin", x="SaleCondition", y="Street", hue="Street") #for categorical data #kind="bar"# kind="swarm"

#Data PreProcessing: Correlation Matrix & HeatMap

a = X_full.corr().nlargest(15,"SalePrice")["SalePrice"] #.index #Top "n" most correlated features
print(a)
plt.figure(figsize=(16,6)) # Make plot size larger
sns.heatmap(X_full[a.index].corr(),vmin=-1,vmax=1,annot=True)


In [ ]:
#Data Exploration: Visualize data 1 of 2

train_features = X_full.columns
Graphing = X_full[['SalePrice']]
# X_full['GarageFinish'].hist()
# type(X_full['SalePrice']) #series
# type(X_full[['SalePrice']]) #dataframe
# X_full[['SalePrice']].plot()
# X_full['SalePrice'].plot())
# X_full[['SalePrice']].plot.area() # or (kind ='area')
# Graphing.plot.bar()
# Graphing.plot.barh()
# Graphing.pie() <- check
# Graphing.plot.box() <- check
# Graphing.plot.hexbin() <- check
# Graphing.plot.kde() <- check
# Graphing.plot.density() <- check
# print(train_features)
# X_full.plot.scatter(x='YearBuilt',y='SalePrice')
# from pandas.plotting import scatter_matrix
# scatter_matrix(X_full[['YearBuilt','GarageArea','YrSold','SalePrice','TotRmsAbvGrd']],
#               alpha=0.2,figsize=(10,8),diagonal='kde') #scatter matrix with density plots in diagonal

In [ ]:
#Data Exploration - Train set: Types of features/columns & Missing features
#Also needs to be performed for Test set data

numbercols = []
objectcols = []
for col in X_full.columns:
    if X_full[col].dtype not in ['object']:
        numbercols.append(col)
    elif X_full[col].dtype in ['object']:
        objectcols.append(col)
        
missingNum = []
missingCat = []

# for element in range(len(numbercols)):
#     print(numbercols[element],X_full[numbercols[element]].dtype)

for col in numbercols:
    if X_full[col].count() < X_full.shape[0]:
        missingNum.append(col)

print(missingNum)

for col in objectcols:
    if X_full[col].count() < X_full.shape[0]:
        missingCat.append(col)

print(missingCat)

# Create replacement approach for each column - for some imputation, for
# others like garages and bedrooms that are optional, can consider inputting
# zero or creating a separate column with a flag 1 or 0

# Example: Imputation with Median for a certain group
# X_full['LotFrontage'] = X_full.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# # Example: Replace Null with 0 = 
# X_full['GarageYrBlt'] = X_full['GarageYrBlt'].fillna(0)


In [ ]:
#Data PreProcessing: Feature Engineering Techniques (numeric)
# https://www.kaggle.com/c/ieee-fraud-detection/discussion/108575#891273

##Feature Profiling (incl BoxCox transformation & visual check)
# skewed = X_full[numbercols].apply(lambda x: skew(x)) # To identify high skew columns (>0.5). Can also use boxcox1p & boxcox_normmax
skewed[skewed>0.5].index

# for col in numbercols: #All NA's must be removed by this point
#     X_full[col] = boxcox1p(X_full[col],boxcox_normmax(X_full[col]+1))
# plt.figure(figsize=(16,9))
# sns.distplot(X_full['LotArea'], kde = False)
# sns.scatterplot(x=X_full['LotArea'], y=X_full['SalePrice'])

#Remove non-informative columns such as those 1 value (or mostly 1 value) across entire column
#as it leads to sparse data set & low quality regression - setting threshold to 90%
#Be sure to adjust the numbercols array accordingly to remove toberemoved values
toberemoved = []
for col in numbercols:
    if X_full[col].value_counts().max() > 0.9*X_full.shape[0]:
        toberemoved.append(col)
print(toberemoved)
X_full.drop(toberemoved,axis=1,inplace = True)

#Correlation Matrix & HeatMap to address collinearity / correlation
a = X_full[numbercols].corr().nlargest(15,"SalePrice")["SalePrice"] #.index #Top "n" most correlated features
print(a)
plt.figure(figsize=(10,10)) # Make plot size larger
sns.heatmap(X_full[a.index].corr(),vmin=-1,vmax=1,annot=True,cmap='BrBG')
plt.savefig('heatmap.png',dpi=300,bbox_inches='tight') #Download as Image
# SalePriceCorr = X_full.corr()[['SalePrice']].sort_values(by='SalePrice',ascending=False)
# plt.figure(figsize=(2,10))
# sns.heatmap(SalePriceCorr,vmin=-1,vmax=1,annot=True)

# Model Validation
# https://www.xenonstack.com/insights/what-is-model-validation-testing/#:~:text=Validation%20%E2%80%93%20The%20process%20of%20developing,and%20doing%20the%20changes%20accordingly.

In [ ]:
cols = [i for i in X_train.isnull().columns if X_train[i].isnull().any()]
reduced_X_train = X_train.drop(cols, axis = 1)
reduced_X_valid = X_valid.drop(cols, axis = 1)

In [3]:
#Data PreProcessing: Data Cleaning

# Convert all train Nominal numeric values to categorical strings, ex. Year
X_full['MSSubClass'] = X_full['MSSubClass'].astype('str')
X_full['YearBuilt'] = X_full['YearBuilt'].astype('str')
X_full['YearRemodAdd'] = X_full['YearRemodAdd'].astype('str')
X_full['GarageYrBlt'] = X_full['GarageYrBlt'].astype('str')
X_full['MoSold'] = X_full['MoSold'].astype('str')
# Convert all test Nominal numeric values to categorical strings, ex. Year
X_test_full['MSSubClass'] = X_test_full['MSSubClass'].astype('str')
X_test_full['YearBuilt'] = X_test_full['YearBuilt'].astype('str')
X_test_full['YearRemodAdd'] = X_test_full['YearRemodAdd'].astype('str')
X_test_full['GarageYrBlt'] = X_test_full['GarageYrBlt'].astype('str')
X_test_full['MoSold'] = X_test_full['MoSold'].astype('str')
X_full.describe(include='all')
X_test_full.describe(include='all')

# Remove rows without a target, separate target from predictors
X_full = X_full.dropna(axis=0, subset=["SalePrice"], inplace = False)
y = X_full["SalePrice"]
X_full.drop(["SalePrice"],axis=1, inplace = True)

# # Keeping things simple, not using "objects" rather just numeric predictors
X = X_full.select_dtypes(exclude=["object"])
X_test = X_test_full.select_dtypes(exclude=["object"])

# # Split validation set from Training data
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 1)

## Imputatation
# Option 1: remove all columns with missing values

# cols = [i for i in X_train.isnull().columns if X_train[i].isnull().any()]
# reduced_X_train = X_train.drop(cols, axis = 1)
# reduced_X_valid = X_valid.drop(cols, axis = 1)

# Option 2: Impute values

from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))
imputed_X_train.columns = X_train.columns #restore column names
imputed_X_valid.columns = X_valid.columns #restore column names

# Option 3: Impute values and mark the rows that were imputed in a separate column (ex. houses with blank under garageArea could possibly not have a garage)

In [ ]:
pct_missing_value_count_by_column = (X_train.isnull().mean()*100) #percent null
missing_value_count_by_column = (X_train.isnull().sum())
print(missing_value_count_by_column[missing_value_count_by_column>0])

print(X_full[X_full.isnull().values])
print(X_full['LotFrontage'].isnull().sum()) #or .any() to check if dataframe has it or not.
# X_train[X_train.isnull().values]
# X_train['LotFrontage'].isnull().sum()

pd.set_option('display.max_columns',None) # to display all columns

In [15]:
# Random Forest
# Import Libraries
from sklearn.ensemble import RandomForestRegressor
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create Rondom Forest classification model object
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    # Train model
    model.fit(X_train,y_train)
    # Predict Output
    prediction = model.predict(X_valid) 
    return mean_absolute_error(y_valid,prediction)

# Model Scoring
sk.cross_val_score(model=model,X=X_train,y=y_train,
                   scoring=‘neg_mean_absolute_error’,cv="") #scoring strategies here https://scikit-learn.org/stable/modules/model_evaluation.html#scoring # Why is regression listed as negative for all scoring types?

In [8]:
print("Answer is:" + str(score_dataset(imputed_X_train,y_train,imputed_X_valid,y_valid)))

Answer is:16657.755650684932


In [16]:
print("Answer is:" + str(score_dataset(reduced_X_train,y_train,reduced_X_valid,y_valid)))

Answer is:16367.230342465757


In [ ]:
# Models ####Be sure to look at model documentation & experiment with various hyperparameters

# Linear Regression
# Import Library & other libraries like pandas, numpy etc.
from sklearn import linear_model
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Load Train and Test datasets - Identify feature and response variable(s) and values must be numeric and numpy arrays
    # Create linear regression object
    linear = linear_model.LinearRegression()
    # Fit/Train the model using training sets & check score ####Need to check the validation set inclusion through args
    linear.fit(X_train, y_train)
    linear.score(X_train, y_train)
    # Equation coefficient & Intercept
    print('Coefficient: \n',linear.coef_)
    print('Intercept: \n',linear.intercept_)
    # Predict Output
    predicted = linear.predict(X_valid)
    return mean_absolute_error(y_valid,predicted)

# Logistic Regression
# Import Library 
from sklearn.linear_model import LogisticRegression
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create logistic regression object
    model = LogisticRegression()
    # Train & score the model using training set
    model.fit(X_train, y_train)
    model.score(X_train, y_train)
    # Equation coefficient & intercept
    print('Coefficient \n':model.coef_)
    print('Intercept \n':model.intercept_)
    # Predict Output
    predicted = model.predict(X_valid)
    return mean_absolute_error(y_valid,predicted)


# Decision Tree
# Import Library 
from sklearn import tree
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create tree object
    model = tree.DecisionTreeClassifier(criterion='gini') # For classification, you can change the algorithm as gini or entropy (information gain); by default it is Gini
    model = tree.DecisionTreeRegressor()
    # Train & score the model
    model.fit(X_train,y_train)
    model.score(X_train,y_train)
    # Predict Output
    prediction = model.predict(X_valid)
    return mean_absolute_error(y_valid,predicted)


# SVM (Support Vector Machine)
# Import Library
from sklearn import svm 
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create SVM classification object
    model = svm.svc()
    # Train & score the model
    model.fit(X_train, y_train)
    model.score(X_train, y_train)
    # Predict Output
    prediction = model.predict(X_valid) 
    return mean_absolute_error(y_valid,predicted)


# Naive Bayes
# Import Libraries
from sklearn.naive_bayes import GaussianNB
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create NB classification object model
    model = GaussianNBare other models for different disbritions: # for continuous values 
    # there are other models for different distributions:Multinomial and Bernoulli Naive Bayes
    # Train model
    model.fit(X_train,y_train)
    # Prediction Output
    prediction = model.predict(X_valid)
    return mean_absolute_error(y_valid,predicted)

# kNN (k-Nearest Neighbors) - Supervised for classification or regression
# Import Libraries
from sklearn.neighbors import KNeighborsClassifier
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create KNeighbors Classifier object model
    model = kNeighborsClassifier(n_neighbors = 6)
    # Train model
    model.fit(X_train,y_train)
    # Predict Output
    prediction = model.predict(X_valid)
    return mean_absolute_error(y_valid,predicted)

# k-Means - Unsupervised for clustering
# Import Libraries
from sklearn.cluster import KMeans
# Create k-Means classifier model object
model = kMeans(n_clusters=3, random_state=0)
# Train model
model.fit(X_train)
# Predict Output
prediction = model.predict(X_valid)

# Random Forest Classification
# Import Libraries
from sklearn.ensemble import RandomForestClassifier
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create Rondom Forest classification model object
    model = RandomForestClassifier()
    # Train model
    model.fit(X_train,y_train)
    # Predict Output
    prediction = model.predict(X_valid) 
    return mean_absolute_error(y_valid,predicted)

# Random Forest Regression
# Import Libraries
from sklearn.ensemble import RandomForestRegressor
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create Rondom Forest classification model object
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    # Train model
    model.fit(X_train,y_train)
    # Predict Output
    prediction = model.predict(X_valid) 
    return mean_absolute_error(y_valid,predicted)

# Dimensionality Reduction
# Import Libraries
from sklearn import decomposition
# The train set contains both X and Y & the test set also contains both X (predictor variables) and Y (target variables)
# Create PCA object
pca = decomposition.PCA(n_components=k)
fa = decomposition.FactorAnalysis()# For factor analysis
# Reduce dimensions of training dataset using PCA
train_reduced = pca.fit_transform(train)
# Reduced the dimension of test set
test_reduced = pca.transform(test)

# Gradient Boosting & AdaBoost
# Import Libraries
from sklearn.ensemble import GradientBoostingClassifier
def score_dataset(X_train,y_train,X_valid,y_valid):
    # Create Gradient Boosting Classifier object
    model = GradientBoostingClassifier.fit()
    # Train the model using the training sets
    model.fit(X_train,y_train)
    # Predict Output
    prediction = model.predict(X_valid)
    return mean_absolute_error(y_valid,predicted)
    


In [ ]:
#Data PreProcessing: Feature Engineering Techniques (Categorical)

## Handling NULLS
# Option 1: Fill NA with the frequent values/mode of equivalent subgroup
# print(X_full['BsmtCond'].value_counts())
# X_full['BsmtCond'] = X_full.groupby('OverallCond')['BsmtCond'].transform(lambda x: x.fillna(x.mode()[0]))
# X_full['BsmtCond'].value_counts()
# sns.scatterplot(x=X_full['SaleCondition'],y=X_full['BsmtCond'])
# sns.catplot(x='BsmtCond',y='OverallCond',data=X_full)
# sns.catplot(x='BsmtCond',y='OverallQual',data=X_full)

# Option 2: Fill NA with a predefined value such as 'None' OR their mode
# print(X_full['Alley'].value_counts())
# X_full['Alley'] = X_full['Alley'].fillna('None')
# X_full['Alley'].value_counts()

# print(X_full['Electrical'].value_counts())
# X_full['Electrical'] = X_full['Electrical'].fillna(X_full['Electrical'].mode()[0])
# X_full['Electrical'].value_counts()

# Option 3: Drop Column - as long as it doesn't have much predictive power
# X = X_train.dtypes=="object"
# print(X)
# drop_X_train = X_train.select_dtypes[exclude='object']
# drop_X_valid = X_valid.select_dtypes[exclude='object']

# Option 4: Label Encoding - for Categorical variables that are ordinal / on a spectrum
# https://www.kaggle.com/alexisbcook/categorical-variables#Three-Approaches

# Option 5: One Hot Encoding - For nominal values - generally <15 types for most models
# https://www.kaggle.com/alexisbcook/categorical-variables#Three-Approaches

##Dropping columns of low value - those with mostly one value
# toremovecat = []
# for col in objectcols: #find out how many cols mostly (90%) same value
#     if X_full[col].value_counts().max() > 0.9*X_full.shape[0]:
#         toremovecat.append(col)
# print(toremovecat)
# X_full.drop(toremovecat,axis=1,inplace=True)


##Make Select Categoricals ordinal/numerical - need to remove NaNs prior
# to_replace={'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'None':0}
# set=['ExterQual', 'ExterCond', 'BsmtQual','BsmtCond', 'HeatingQC',
#         'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']
# for col in set:
#     X_full[col].replace(to_replace,replacement,inplace=True)

# map2 = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'None': 0}
# ['BsmtExposure']

# map3 = {'GLQ': 4,'ALQ': 3,'BLQ': 2,'Rec': 3,'LwQ': 2,'Unf': 1,'None': 0}
# set3 = ['BsmtFinType1', 'BsmtFinType2']

# map4 = {'Y': 1, 'N': 0}
# ['CentralAir']

# map5 = {'Typ': 3, 'Min1': 2.5, 'Min2': 2, 'Mod': 1.5, 'Maj1': 1, 'Maj2': 0.5, 'Sev': 0, 'Sal': 0}
# ['Functional'] 

##Orginal Categories can be combined such as Garage Quality (times) Garage Condition etc.
##if it creates greater impact than having either one separately or both

##One-hot encode the rest of the nominal categorical variables using Pandas get_dummies function
X_full = pd.get_dummies(X_full).reset_index(drop=True)
a=X_full.columns
print(a)

## Setup Pipeline for preprocessing of numerical & categorical variables
# https://www.kaggle.com/alexisbcook/pipelines